In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 
import seaborn as sns
from matplotlib import pyplot as plt

from IPython.core.display import display, HTML
from PIL import Image
from io import BytesIO
import base64

import os
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Input, BatchNormalization, ReLU, AveragePooling2D
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, UpSampling2D
from tensorflow.keras.layers import Lambda, Subtract, Add
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import np_utils

%matplotlib inline

In [ ]:
# set variables 
main_folder = '../input/celeba-dataset/'
images_folder = main_folder + 'img_align_celeba/img_align_celeba/'
EXAMPLE_PIC = images_folder + '000506.jpg'

In [ ]:
# import the data set that include the attribute for each picture
df_attr = pd.read_csv(main_folder + 'list_attr_celeba.csv')
df_attr.set_index('image_id', inplace=True)
df_attr.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0
df_attr.shape

In [ ]:
glass = df_attr["Eyeglasses"]
# In gender array 0-no glass while 1-glass
glass.head(5)

In [ ]:
df_partition = pd.read_csv(main_folder + 'list_eval_partition.csv')
# df_partition.head()

In [ ]:
df_partition['partition'].value_counts().sort_index()

In [ ]:
df_partition.set_index('image_id', inplace=True)
df_par_attr = df_partition.join(glass, how='inner')
df_par_attr.head()

In [ ]:
def load_reshape_img(fname):
    x = cv2.imread(fname)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (64,64)).astype('float32') / 255.
    x = x.reshape((1,) + x.shape)

    return x


def generate_df(partition, attr, num_samples):
    '''
    partition
        0 -> train
        1 -> validation
        2 -> test
    
    '''
    
    df_ = df_par_attr[(df_par_attr['partition'] == partition) 
                           & (df_par_attr[attr] == 0)].sample(int(num_samples/2))
    df_ = pd.concat([df_,
                      df_par_attr[(df_par_attr['partition'] == partition) 
                                  & (df_par_attr[attr] == 1)].sample(int(num_samples/2))])

    # for Train and Validation
    if partition != 2:
        x_ = np.array([load_reshape_img(images_folder + fname) for fname in df_.index])
        x_ = x_.reshape(x_.shape[0], 64, 64, 3)
        y_ = np_utils.to_categorical(df_[attr],2)
    # for Test
    else:
        x_ = []
        y_ = []

        for index, target in df_.iterrows():
            im = cv2.imread(images_folder + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_WIDTH, IMG_HEIGHT)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            x_.append(im)
            y_.append(target[attr])

    return x_, y_

In [ ]:
TRAINING_SAMPLES = 50000
VALIDATION_SAMPLES = 500
IMG_WIDTH = 64
IMG_HEIGHT = 64

x_train, y_train = generate_df(0, 'Eyeglasses', TRAINING_SAMPLES)
# x_valid, y_valid = generate_df(1, 'Eyeglasses', VALIDATION_SAMPLES)

In [ ]:
count = 0
for i in range(5000):
    if (y_train[i]==np.array([1,0])).all():
        count += 1
print(x_train.shape)
count

# **CLF**

In [ ]:
input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)
kernel_size = 5
gamma_init = tf.random_normal_initializer(1., 0.02)


class glass_clf(Model):
    def __init__(self, **kwargs):
      super(glass_clf, self).__init__(**kwargs)
      self.f2l = Sequential(
          [
           Input(shape=input_shape, name='classifier_input'),
           Conv2D(filters=64, kernel_size=kernel_size, strides=2, padding='same'),
           BatchNormalization(gamma_initializer=gamma_init, trainable=True),
           ReLU(),
           Conv2D(filters=128, kernel_size=kernel_size, strides=2, padding='same'),
           BatchNormalization(gamma_initializer=gamma_init, trainable=True),
           ReLU(),
           Conv2D(filters=256, kernel_size=kernel_size, strides=2, padding='same'),
           BatchNormalization(gamma_initializer=gamma_init, trainable=True),
           ReLU(),
           Flatten(),
           Dense(512, activation='relu'),
          ]
      )
      self.last = Sequential(
          [
           Input(shape=(512,)),
#            Dense(units=2, activation = 'softmax'),
           Dense(units=1, activation = 'sigmoid'),
          ]
      )
    
    def call(self, inputs, training=None):
      return self.last(self.f2l(inputs))

clf = glass_clf()
clf.build(input_shape=(None, IMG_WIDTH, IMG_HEIGHT, 3))
clf.summary()

# **VAE**

In [ ]:
# create a sampling layer
from tensorflow.keras.layers import Layer
class reparameterize(Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        mean, logvar = inputs
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        eps = tf.keras.backend.random_normal(shape=(batch, dim))
        return eps * tf.exp(logvar * .5) + mean

In [ ]:
# encoder
input_shape = (64, 64, 3)
kernel_size = 5
latent_dim = 512
# latent_dim = 256
image_size = 64 # square, original to be resized
crop_size = 148 # center crop to this size per dimension
batch_size = 128
gamma_init = tf.random_normal_initializer(1., 0.02)

# First build the Encoder Model part
inputs = Input(shape=input_shape, name='encoder_input')
x = Conv2D(filters=64, kernel_size=kernel_size, strides=2, padding='same', name='conv1')(inputs)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2D(filters=128, kernel_size=kernel_size, strides=2, padding='same', name='conv2')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2D(filters=256, kernel_size=kernel_size, strides=2, padding='same', name='conv3')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2D(filters=512, kernel_size=kernel_size, strides=2, padding='same', name='conv4')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)

shape = K.int_shape(x)

x = Flatten()(x)
mean = Dense(latent_dim, name="z_mean")(x)
logvar = Dense(latent_dim, name="z_log_var")(x)
z = reparameterize()([mean, logvar])

# Instantiate Encoder Model
encoder = Model(inputs, [z, mean, logvar], name='encoder')
encoder.summary()

In [ ]:
# decoder
latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(shape[1] * shape[2] * shape[3] * 2)(latent_inputs)
x = Reshape((shape[1]*2, shape[2]*2, shape[3]//2), name='h0_reshape')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)

x = Conv2DTranspose(filters=256, kernel_size=5, strides=2, padding='same')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2DTranspose(filters=128, kernel_size=5, strides=2, padding='same')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2DTranspose(filters=64, kernel_size=5, strides=2, padding='same')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2DTranspose(filters=3, kernel_size=5, strides=1, padding='same')(x)
outputs = Activation('tanh', name='decoder_output')(x)

# Instantiate Decoder Model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

In [ ]:
class VAE(Model):
    def __init__(self, encoder, latent_dim, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.cf_layer = Dense(units=latent_dim, kernel_initializer=tf.constant_initializer(np.eye(latent_dim)))
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z, mean, logvar = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 64 * 64
            kl_loss = 1 + logvar - tf.square(mean) - tf.exp(logvar)
            kl_loss = tf.reduce_mean(kl_loss)
#             kl_loss = tf.reduce_sum(kl_loss)
            kl_loss *= -0.5
#             LOSS_FACTOR = 10000
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, inputs):
      z, mean, logvar = encoder(inputs)
      return decoder(z)

    def cf(self, inputs):
      z, mean, logvar = encoder(inputs)
      return decoder(self.cf_layer(z))

    @tf.function
    def sample(self, eps=None):
      if eps is None:
        eps = tf.random.normal(shape=(100, latent_dim))
      return tf.sigmoid(self.decoder(eps))

# Instantiate vae
vae = VAE(encoder, latent_dim, decoder)

In [ ]:
# load pretrained weights
vae.load_weights('../input/cgen-pretrain-vae/CELEBVAE_45.tf')
# vae.load_weights('../input/cage-pretrain-vae-256/CELEBVAE_20_256.tf')
clf.load_weights('../input/glassclf-sigmoid/glass_cf_15_sigmoid.tf')

# **simple test**

In [ ]:
# simple test
test_img = load_reshape_img('../input/celeba-dataset/img_align_celeba/img_align_celeba/000144.jpg')
plt.imshow(test_img[0])

In [ ]:
test_result = vae(test_img)
test_clf = clf(test_img)
print(test_clf)
plt.imshow(test_result[0])

# **train cgen**

In [ ]:
test = y_train
# test_result = [np.argmax(one_hot)for one_hot in test]

In [ ]:
# plt.imshow(x_train[6500])

In [ ]:
from tqdm import tqdm
y_interger = [np.argmax(one_hot)for one_hot in y_train]

train_dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_interger))
                     .shuffle(len(x_train)).batch(batch_size))

def step(vae, classifier, optimizer, optimizer2, batch_size, alpha):
    vae.cf_layer.trainable = True
    total = 0
    vae_loss = 0
    clf_loss = 0
    disc_total = 0
    batch_num = 0
    for batch in tqdm(train_dataset):
        with tf.GradientTape() as dec_tape, tf.GradientTape() as disc_tape:
            batch_num += 1
            counterfactual = vae.cf(batch[0])
            # for d_g
            reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(batch[0], counterfactual))
            d_g = reconstruction_loss

            index_ori = np.where(K.eval(batch[1]) == 0)
            index_target = np.where(K.eval(batch[1]) == 1)
            target_tensor = np.ones([len(batch[0]), 1])
            target_tensor[index_ori[0]] = target_tensor[index_ori[0]]* 0.1
            target_tensor[index_target[0]] = target_tensor[index_target[0]]* 0.9

            # for d_c
            logits = classifier(counterfactual)
            disc_loss = tf.reduce_mean(tf.reduce_mean(tf.keras.losses.binary_crossentropy(target_tensor, logits)))
            target_tensor = np.ones([len(batch[0]), 1])
            target_tensor[index_ori[0]] = target_tensor[index_ori[0]]* 0.9
            target_tensor = tf.convert_to_tensor(target_tensor, dtype=tf.float32)
            d_c = tf.reduce_mean(tf.keras.losses.binary_crossentropy(target_tensor, logits))
            total_loss = (1. - alpha)*d_g + alpha * d_c
            total += total_loss
            vae_loss += d_g
            clf_loss += d_c
            disc_total += disc_loss
            # clf_loss += d_c*len(batch)
        dec_gradients = dec_tape.gradient(total_loss, vae.cf_layer.trainable_weights)
        disc_gradients = disc_tape.gradient(disc_loss, classifier.trainable_weights)
        optimizer.apply_gradients(zip(dec_gradients, vae.cf_layer.trainable_weights))
        optimizer2.apply_gradients(zip(disc_gradients, classifier.trainable_weights))
    return [total/batch_num, vae_loss/batch_num, clf_loss/batch_num, disc_total/batch_num]

In [ ]:
test_img1 = load_reshape_img('../input/celeba-dataset/img_align_celeba/img_align_celeba/199998.jpg')
test_img2 = load_reshape_img('../input/celeba-dataset/img_align_celeba/img_align_celeba/201557.jpg')
test_img3 = load_reshape_img('../input/celeba-dataset/img_align_celeba/img_align_celeba/200003.jpg')
test_img4 = load_reshape_img('../input/celeba-dataset/img_align_celeba/img_align_celeba/200233.jpg')

test_img1 = np.concatenate([test_img1, test_img2])
test_img1 = np.concatenate([test_img1, test_img3])
test_img = np.concatenate([test_img1, test_img4])



# valid_dataset = (tf.data.Dataset.from_tensor_slices(x_valid).batch(batch_size))
# for test_batch in valid_dataset.take(1):
#     test_img = test_batch[0:4, :, :, :]

In [ ]:
def generate_and_save_cf(model, test_sample):
    result = model.cf(test_sample)
    fig = plt.figure(figsize=(5, 5))
    reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(result, test_sample))
    print(reconstruction_loss)
    for i in range(result.shape[0]):
        plt.subplot(2, 2, i + 1)
        plt.imshow(result[i, :, :, :])
        plt.axis('off')
    plt.show()

generate_and_save_cf(vae, test_img)

In [ ]:
beta_1 = 0.5
vae = VAE(encoder, latent_dim, decoder)
vae.load_weights('../input/cgen-pretrain-vae/CELEBVAE_45.tf')
clf.load_weights('../input/glassclf-sigmoid/glass_cf_15_sigmoid.tf')

optimizer=Adam(beta_1=beta_1)
optimizer2 = Adam()
batch_size = 128
total_loss = []
vae_loss = []
clf_loss = []
dis_loss = []

train_cycles = 200
for train_cycle in range(train_cycles):
    print("Cgen train cycle:", train_cycle, train_cycles)
    # cGen training
    cgen_loss = step(vae, clf, optimizer, optimizer2, batch_size, 0.01)
    print('cgen_loss: {a}, vae_loss: {b}, clf_loss: {c}, disc_loss: {d}'.format(a=cgen_loss[0], b=cgen_loss[1], c=cgen_loss[2], d=cgen_loss[-1]))
 
    generate_and_save_cf(vae, test_img)
    a = K.eval(cgen_loss[0])
    b = K.eval(cgen_loss[1])
    c = K.eval(cgen_loss[2])
    d = K.eval(cgen_loss[3])
    total_loss.append(a)
    vae_loss.append(b)
    clf_loss.append(c)
    dis_loss.append(d)

In [ ]:
fig_1 = plt.figure(figsize=(8, 4))
ax_1 = fig_1.add_subplot(111)
ax_1.plot(np.arange(1, len(total_loss)) * 1, total_loss[1:], label='total')
ax_1.plot(np.arange(1, len(vae_loss)) * 1, vae_loss[1:], label='first term')
# ax_1.plot(np.arange(1, len(clf_loss)) * 1, clf_loss[1:], label='second term')
# ax_1.plot(np.arange(1, len(dis_loss)) * 1, dis_loss[1:], label='disc term')
ax_1.legend(loc=0)
ax_1.set_xlabel('Epoch number')
ax_1.set_ylabel('binary cross entropy')

In [ ]:
total_loss = np.array(total_loss)
vae_loss = np.array(vae_loss)
clf_loss = np.array(clf_loss)

np.save('total.npy',total_loss)
np.save('first.npy', vae_loss)
np.save('sec.npy',clf_loss)

In [ ]:
vae.save_weights('cgen-512-0.01-200/model.tf')